In [1]:
import re
import nltk
import pandas as pd
import numpy as np

## Divide into training and testing corpus

In [2]:
training_files = [
    '../files/allan_poe/berenice.txt',
    '../files/allan_poe/decenso_al_maelstron.txt',
    '../files/allan_poe/el_barril_de_amontillado.txt',
    '../files/allan_poe/el_diablo_en_el_campanario.txt',
    '../files/allan_poe/el_engaño_del_globo.txt',
    '../files/allan_poe/el_entierro_prematuro.txt',
    '../files/allan_poe/el_hombre_de_la_multitud.txt',
    '../files/allan_poe/el_retrato_oval.txt',
    '../files/allan_poe/gato_negro.txt',
    '../files/allan_poe/la_caída_de_la_casa_usher.txt',
    '../files/allan_poe/la_esfinje.txt',
    '../files/allan_poe/la_máscara_de_la_muerte_roja.txt',
    '../files/allan_poe/lady_ligeia.txt',

    '../Files/cien_soledad.txt',
    '../Files/el_principito.txt',
    '../Files/la_isla_del_tesoro.txt',
]
testing_files = [
    '../files/allan_poe/metzengerstein.txt',
    '../files/allan_poe/morerlla.txt',
    '../files/allan_poe/william_wilson.txt',
]

In [3]:
# Create training corpus
texts = []
for file_name in training_files:
    file = open(file_name, 'r')
    texts += [file.read()]
    file.close()
texts = pd.Series(texts, index=training_files)
training_text = ''.join(texts)

texts

../files/allan_poe/berenice.txt                        La desdicha es diversa. La desgracia cunde mul...
../files/allan_poe/decenso_al_maelstron.txt            Habíamos alcanzado la cumbre del despeñadero m...
../files/allan_poe/el_barril_de_amontillado.txt        Lo mejor que pude había soportado las mil inju...
../files/allan_poe/el_diablo_en_el_campanario.txt      Todos saben de una manera vaga que el lugar má...
../files/allan_poe/el_engaño_del_globo.txt             ¡Asombrosas noticias por expreso, vía Norfolk!...
../files/allan_poe/el_entierro_prematuro.txt           Hay ciertos temas de interés absorbente, pero ...
../files/allan_poe/el_hombre_de_la_multitud.txt        Con razón se ha dicho de cierto libro alemán q...
../files/allan_poe/el_retrato_oval.txt                 El castillo en el cual mi criado se le había o...
../files/allan_poe/gato_negro.txt                      No espero ni pido que alguien crea en el extra...
../files/allan_poe/la_caída_de_la_casa_usher.txt       

In [4]:
# Create testing corpus
texts = []
for file_name in testing_files:
    file = open(file_name, 'r')
    texts += [file.read()]
    file.close()
texts = pd.Series(texts, index=testing_files)
testing_text = ''.join(texts)
texts

../files/allan_poe/metzengerstein.txt    El horror y la fatalidad han estado al acecho ...
../files/allan_poe/morerlla.txt          Consideraba yo a mi amiga Morella con un senti...
../files/allan_poe/william_wilson.txt    Permitan que, por el momento, me presente como...
dtype: object

## Tokenization

In [5]:
text = training_text.lower()
text = re.sub(r'[-)(\s«»"]+', ' ', text) # Ignored characters
text = re.sub(r'\.+\s*', '<s>', text) # Replace end of sentences
text = re.sub(r'\d+', '<n>', text) # Unify numbers

tokens = re.finditer(r'[\wáéíóúñ]+|<[sn]>|[,¿?!¡;:]', text)
tokens = pd.Series([m.group(0) for m in tokens])

tokens

0               la
1         desdicha
2               es
3          diversa
4              <s>
            ...   
309510        ocho
309511      piezas
309512          de
309513           á
309514        ocho
Length: 309515, dtype: object

# 
## Unigrams

In [6]:
unigrams, counts = np.unique(tokens, return_counts=True)
unigrams = pd.DataFrame(counts, index=unigrams, columns=['count'])
unigrams = unigrams.sort_values(by='count', ascending=False)

unigrams['prob'] = unigrams['count'] / unigrams['count'].sum()
unigrams['cumsum'] = unigrams['prob'].cumsum()

unigrams

,count,prob,cumsum
de,16470,0.053212,0.053212
",",12800,0.041355,0.094567
la,10804,0.034906,0.129474
que,9581,0.030955,0.160428
y,9006,0.029097,0.189526
...,...,...,...
infundado,1,0.000003,0.999987
infundieron,1,0.000003,0.999990
infundir,1,0.000003,0.999994
infundirle,1,0.000003,0.999997


## Bigrams

In [7]:
bigrams = np.fromiter(nltk.ngrams(tokens, 2), dtype=('<U18, <U18'))
bigrams, counts = np.unique(bigrams, return_counts=True, axis=0)

bigrams = pd.MultiIndex.from_tuples(list(bigrams), names=['w1', 'w2'])

bigrams = pd.DataFrame(counts, index=bigrams, columns=['count'])
bigrams = bigrams.sort_values(by='count', ascending=False)
bigrams['prfix-sum'] = bigrams['count'].groupby(level='w1').transform(np.sum)

bigrams['prob'] = bigrams['count'] / bigrams['prfix-sum']
bigrams['cumsum'] = bigrams.prob.groupby(level=['w1']).cumsum()

bigrams

count  prfix-sum      prob    cumsum
w1         w2                                           
de         la        2369      16470  0.143837  0.143837
,          y         2067      12800  0.161484  0.161484
en         el        1258       7421  0.169519  0.169519
           la        1118       7421  0.150654  0.320172
de         los        856      16470  0.051973  0.195811
...                   ...        ...       ...       ...
klim       ,            1          1  1.000000  1.000000
kircher    y            1          1  1.000000  1.000000
kilómetros más          1          6  0.166667  1.000000
casi       desmayo      1        196  0.005102  1.000000
útiles     y            1          5  0.200000  1.000000

[137958 rows x 4 columns]

## Trigrams

In [8]:
trigrams = np.fromiter(nltk.ngrams(tokens, 3), dtype=('<U18, <U18, <U18'))
trigrams, counts = np.unique(trigrams, return_counts=True, axis=0)

trigrams = pd.DataFrame(trigrams)
trigrams['w1'] = trigrams[['f0', 'f1']].apply(" ".join, axis=1)
trigrams['w2'] = trigrams[['f2']]
trigrams = pd.MultiIndex.from_frame(trigrams[['w1', 'w2']])

trigrams = pd.DataFrame(counts, index= trigrams, columns=['count'])
trigrams = trigrams.sort_values(by='count', ascending=False)
trigrams['prfix-sum'] = trigrams['count'].groupby(level='w1').transform(np.sum)

trigrams['prob'] = trigrams['count'] / trigrams['prfix-sum']
trigrams['cumsum'] = trigrams.prob.groupby(level=['w1']).cumsum()

trigrams

count  prfix-sum      prob    cumsum
w1                w2                                             
coronel aureliano buendía      191        198  0.964646  0.964646
josé arcadio      buendía      162        374  0.433155  0.433155
sin embargo       ,            137        210  0.652381  0.652381
, y               que          133       2067  0.064344  0.064344
el coronel        aureliano    130        196  0.663265  0.663265
...                            ...        ...       ...       ...
encerró con       josé           1          5  0.200000  0.400000
                  llave          1          5  0.200000  0.600000
                  su             1          5  0.200000  0.800000
                  tranca         1          5  0.200000  1.000000
útiles y          por            1          1  1.000000  1.000000

[248688 rows x 4 columns]

## 4-grams

In [9]:
fourgrams = np.fromiter(nltk.ngrams(tokens, 4), dtype=('<U18, <U18, <U18, <U18'))
fourgrams, counts = np.unique(fourgrams, return_counts=True, axis=0)

fourgrams = pd.DataFrame(fourgrams)
fourgrams['w1'] = fourgrams[['f0', 'f1', 'f2']].apply(" ".join, axis=1)
fourgrams['w2'] = fourgrams[['f3']]
fourgrams = pd.MultiIndex.from_frame(fourgrams[['w1', 'w2']])

fourgrams = pd.DataFrame(counts, index= fourgrams, columns=['count'])
fourgrams = fourgrams.sort_values(by='count', ascending=False)
fourgrams['prfix-sum'] = fourgrams['count'].groupby(level='w1').transform(np.sum)

fourgrams['prob'] = fourgrams['count'] / fourgrams['prfix-sum']
fourgrams['cumsum'] = fourgrams.prob.groupby(level=['w1']).cumsum()

fourgrams

,,count,prfix-sum,prob,cumsum
w1,w2,,,,
el coronel aureliano,buendía,126,130,0.969231,0.969231
", sin embargo",",",71,72,0.986111,0.986111
<s> sin embargo,",",60,60,1.000000,1.000000
"remedios , la",bella,53,54,0.981481,0.981481
santa sofía de,la,53,53,1.000000,1.000000
...,...,...,...,...,...
el pelo hacia,adentro,1,1,1.000000,1.000000
el pelo hirsuto,y,1,1,1.000000,1.000000
el pelo mojado,y,1,1,1.000000,1.000000


# Generate text
## Unigrams

In [10]:
n_sentences = 5
words = []
while n_sentences > 0:
    p = np.random.rand()
    row_struct = unigrams[unigrams['cumsum'] > p].iloc[0]
    words += [row_struct.name]
    if row_struct.name == '<s>': 
        n_sentences-=1

' '.join(words)

'ratas la boa ella cuando los y y sentidos , oyó yacía palabras una pasillo la sobre , respondí fría tan situará sucedido confuso corazones es abominable haciendo después el a sus a había la días si le estirada estaba plausible bordando á buena lo de mundo miedo leído aún los los la de desagradar olas <s> su tanto he imágenes a eminencia tuvo conservador macetas puedes la de no xv virginia que de al que sus parece la declaración y dijo modo eufemismos dormida para rebeca un del <s> está manos el dió el tenderme asesores de espejos ahogándose veces corazón menor había y haya lado lo había john casa á una ya eso diversos mortal volvió , edificio que ; las verdad con cintura en con paredes no de en sido trapos por duda sin cachacos <s> mandó serpiente inocente y á irreprimibles calma atado , vds puerta invisible gracias era mí hijos fanatismo trabajo para <s> general estaba al torrecilla <s>'

## Bigrams

In [11]:
n_sentences = 3
words = ['<s>']
while n_sentences > 0:
    p = np.random.rand()
    prev_word = words[-1]
    row_struct = bigrams[bigrams['cumsum'] > p].loc[prev_word].iloc[0]
    words += [row_struct.name]
    if row_struct.name == '<s>': 
        n_sentences-=1

' '.join(words)

'<s> la marea , descargué un baile , dejándola enredarse en el menor grado de canciones de aquí amigo <s> iluminaba con tranquilo ? aureliano buendía <s> el fulgor tan vigorosos como su pañuelo alrededor del mal humor á fusilar <s>'

## Trigrams

In [12]:
n_sentences = 5

# Generarte from bigrams
words = words[:2]
while n_sentences > 0:
    p = np.random.rand()
    prev_word = ' '.join(words[-2:])
    row_struct = trigrams[trigrams['cumsum'] > p].loc[prev_word].iloc[0]
    words += [row_struct.name]
    if row_struct.name == '<s>': 
        n_sentences-=1

' '.join(words)

'<s> la casa de los re cuerdos <s> mientras duró el almuerzo y apagó su farol mientras el coronel aureliano buendía , le miré fijamente a la parte del camino que nos había puesto el hermoso traje de seda natural y con el cuello del frasco con la semana , y aun a pleno día <s> y el baúl de los sacos de lona de velero y reforzado con un traje desde los primeros en proporcionar al público : el hombre alto de manera de pronunciarlas con que no hay nada replicó el doctor y el más profundo interés por las noches de la isla nos acercábamos al borde de la isla y una mirada , antes de que nos envolvía , todo asumió una actitud tan infantil frente a su reciente aplicación <s> una noche , por supuesto , las palomas , no puede dar continuó el capitán smollet apuntando hacia él se mantuvo en contacto con el dogal de seda , el de más edad en calidad de su desmandada voracidad , de gentil tarea : la creciente generosidad de los protagonistas <s> pero a pesar de su vida desordenada , todo el alcance ,

## Four-Grams

In [13]:
n_sentences = 3

# Generarte from trigrams
words = words[:3]
while n_sentences > 0:
    p = np.random.rand()
    prev_word = ' '.join(words[-3:])
    row_struct = fourgrams[fourgrams['cumsum'] > p].loc[prev_word].iloc[0]
    words += [row_struct.name]
    if row_struct.name == '<s>': 
        n_sentences-=1
' '.join(words)

'<s> la casa se estaba llenando de duendes <s> era como si el tiempo diera vueltas en redondo y hubiéramos vuelto al principio <s> llovía con intensidad , el aire se impregnó de una fragancia mortal <s>'

# Zero words
Replaze the less frecuent word by the OOV token <UNK>
## Unigrams

In [14]:
lessFrec = unigrams.iloc[-1]
unigrams = unigrams.rename(index={lessFrec.name: '<UNK>'})
unigrams

,count,prob,cumsum
de,16470,0.053212,0.053212
",",12800,0.041355,0.094567
la,10804,0.034906,0.129474
que,9581,0.030955,0.160428
y,9006,0.029097,0.189526
...,...,...,...
infundado,1,0.000003,0.999987
infundieron,1,0.000003,0.999990
infundir,1,0.000003,0.999994
infundirle,1,0.000003,0.999997


## Bigrams

In [15]:
index_np = bigrams.index.to_numpy()
index_np[-1] = ('<UNK>', '<s>')
bigrams.index = pd.MultiIndex.from_tuples(index_np)
bigrams

count  prfix-sum      prob    cumsum
de         la        2369      16470  0.143837  0.143837
,          y         2067      12800  0.161484  0.161484
en         el        1258       7421  0.169519  0.169519
           la        1118       7421  0.150654  0.320172
de         los        856      16470  0.051973  0.195811
...                   ...        ...       ...       ...
klim       ,            1          1  1.000000  1.000000
kircher    y            1          1  1.000000  1.000000
kilómetros más          1          6  0.166667  1.000000
casi       desmayo      1        196  0.005102  1.000000
<UNK>      <s>          1          5  0.200000  1.000000

[137958 rows x 4 columns]

## Trigrams

In [16]:
index_np = trigrams.index.to_numpy()
index_np[-1] = ('<UNK>', '<s>')
trigrams.index = pd.MultiIndex.from_tuples(index_np)
trigrams

count  prfix-sum      prob    cumsum
coronel aureliano buendía      191        198  0.964646  0.964646
josé arcadio      buendía      162        374  0.433155  0.433155
sin embargo       ,            137        210  0.652381  0.652381
, y               que          133       2067  0.064344  0.064344
el coronel        aureliano    130        196  0.663265  0.663265
...                            ...        ...       ...       ...
encerró con       josé           1          5  0.200000  0.400000
                  llave          1          5  0.200000  0.600000
                  su             1          5  0.200000  0.800000
                  tranca         1          5  0.200000  1.000000
<UNK>             <s>            1          1  1.000000  1.000000

[248688 rows x 4 columns]

## 4-grams

In [17]:
index_np = fourgrams.index.to_numpy()
index_np[-1] = ('<UNK>', '<s>')
fourgrams.index = pd.MultiIndex.from_tuples(index_np)
fourgrams

,,count,prfix-sum,prob,cumsum
el coronel aureliano,buendía,126,130,0.969231,0.969231
", sin embargo",",",71,72,0.986111,0.986111
<s> sin embargo,",",60,60,1.000000,1.000000
"remedios , la",bella,53,54,0.981481,0.981481
santa sofía de,la,53,53,1.000000,1.000000
...,...,...,...,...,...
el pelo hacia,adentro,1,1,1.000000,1.000000
el pelo hirsuto,y,1,1,1.000000,1.000000
el pelo mojado,y,1,1,1.000000,1.000000
el pelo pintado,de,1,1,1.000000,1.000000
